## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description
0,0,Mataura,-46.1927,168.8643,35.47,99,100,0.94,NZ,overcast clouds
1,1,Pevek,69.7008,170.3133,41.58,76,24,4.74,RU,few clouds
2,2,Hanno,35.8519,139.3181,77.67,96,90,10.36,JP,heavy intensity rain
3,3,Aksha,50.2814,113.2867,53.46,94,5,4.36,RU,clear sky
4,4,Petropavlovsk-Kamchatskiy,53.0452,158.6483,45.19,100,75,4.47,RU,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description
2,2,Hanno,35.8519,139.3181,77.67,96,90,10.36,JP,heavy intensity rain
6,6,Gwadar,25.1216,62.3254,80.62,85,99,5.66,PK,overcast clouds
9,9,Faanui,-16.4833,-151.7500,78.06,71,25,8.01,PF,scattered clouds
11,11,Butaritari,3.0707,172.7902,82.22,69,15,14.38,KI,few clouds
18,18,Bengkulu,-3.8004,102.2655,75.76,91,11,3.60,ID,few clouds
24,24,Atuona,-9.8000,-139.0333,76.87,83,88,16.15,PF,light rain
26,26,Nanlong,31.3533,106.0631,75.52,92,100,2.66,CN,overcast clouds
29,29,Bucerias,20.7500,-105.3333,87.98,70,20,9.22,MX,few clouds
34,34,Frutal,-20.0247,-48.9406,82.58,25,12,5.77,BR,few clouds
35,35,Albany,42.6001,-73.9662,87.15,67,99,1.01,US,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows. 
preferred_cities_df.count()

City_ID        256
City           256
Lat            256
Lng            256
Max Temp       256
Humidity       256
Cloudiness     256
Wind Speed     256
Country        256
Description    256
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# preferred_cities_df = preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Hanno,JP,77.67,heavy intensity rain,35.8519,139.3181,
6,Gwadar,PK,80.62,overcast clouds,25.1216,62.3254,
9,Faanui,PF,78.06,scattered clouds,-16.4833,-151.7500,
11,Butaritari,KI,82.22,few clouds,3.0707,172.7902,
18,Bengkulu,ID,75.76,few clouds,-3.8004,102.2655,
24,Atuona,PF,76.87,light rain,-9.8000,-139.0333,
26,Nanlong,CN,75.52,overcast clouds,31.3533,106.0631,
29,Bucerias,MX,87.98,few clouds,20.7500,-105.3333,
34,Frutal,BR,82.58,few clouds,-20.0247,-48.9406,
35,Albany,US,87.15,overcast clouds,42.6001,-73.9662,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
# Create a dataframe of just the missing hotels
missing_hotels = hotel_df[hotel_df["Hotel Name"] == '']

# get the index of the missing hotels
index = missing_hotels.index

# drop the rows with out a hotel name using index.tolist() (list of the indexes)
clean_hotel_df = hotel_df.drop(index.tolist())
index.tolist()

[35, 184, 222, 258, 319, 385, 593, 608, 624, 695]

In [11]:
clean_hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Hanno,JP,77.67,heavy intensity rain,35.8519,139.3181,Hotel Heritage Hanno
6,Gwadar,PK,80.62,overcast clouds,25.1216,62.3254,Sadaf Resort
9,Faanui,PF,78.06,scattered clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
11,Butaritari,KI,82.22,few clouds,3.0707,172.7902,Isles Sunset Lodge
18,Bengkulu,ID,75.76,few clouds,-3.8004,102.2655,Grage Hotel Bengkulu
...,...,...,...,...,...,...,...
693,Ranot,TH,81.23,broken clouds,7.7777,100.3213,Pakbanglake resort ปากบางเลค รีสอร์ท
696,Praya,CV,79.34,broken clouds,14.9215,-23.5087,LT Aparthotel
699,Rosarito,MX,88.03,clear sky,32.3333,-117.0333,Hotel Festival Plaza Rosarito
701,Karaton,ID,78.96,few clouds,-6.3424,106.1045,Parkir Mobil Wisata CAS Water Park


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# Save the fig as WeatherPy_vacation_map.png
